# Padding & Stride

In [1]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import math

In [2]:
!rm -rf ./imitools && git clone https://github.com/GDi4K/imitools.git
import imitools as I

Cloning into 'imitools'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 109 (delta 57), reused 31 (delta 13), pack-reused 0
Receiving objects: 100% (109/109), 4.44 MiB | 12.47 MiB/s, done.
Resolving deltas: 100% (57/57), done.


## Padding

With padding, we are trying to add some marging to our image.

In [3]:
X = torch.ones(4, 4)
X

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [4]:
def apply_pad(tensor, n_pad=1):
    tw, th = tensor.shape
    y = torch.zeros(tw + n_pad * 2, th + n_pad *2)
    y[n_pad:n_pad + tw, n_pad:n_pad + th] = tensor
    return y
    
apply_pad(X, 1)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 0.],
        [0., 1., 1., 1., 1., 0.],
        [0., 1., 1., 1., 1., 0.],
        [0., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [5]:
def run_conv(X, K, n_pad=0):
    assert len(X.shape) == 2, "Only 2D tensors are supported"
    X = apply_pad(X, n_pad)
    kw, kh = K.shape
    xw, xh = X.shape
    yw = xw - (kw - 1)
    yh = xh - (kh - 1)
    
    Y = torch.zeros(yw, yh)
    for u in range(yw):
        for v in range(yh):
            selection = X[u:u+kw, v:v+kh]
            Y[u, v] = (selection*K).sum()
    return Y

In [6]:
image = torch.ones(6, 6)
image[:, 2:4] = 0
image

tensor([[1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.]])

In [7]:
run_conv(image, torch.FloatTensor([[1, -1]]))

tensor([[ 0.,  1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.,  0.]])

In [8]:
run_conv(image, torch.FloatTensor([[1, -1]]), n_pad=1)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [-1.,  0.,  1.,  0., -1.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [9]:
run_conv(image, torch.FloatTensor([[1, -1]]), n_pad=2)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  1.,  0., -1.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  1.,  0., -1.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  1.,  0., -1.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  1.,  0., -1.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  1.,  0., -1.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  1.,  0., -1.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

**Yep. It does what we wanted**

## Striding

Here we jump by these amount.

In [10]:
def run_conv(X, K, n_pad=0, n_stride=1):
    assert len(X.shape) == 2, "Only 2D tensors are supported"
    X = apply_pad(X, n_pad)
    kw, kh = K.shape
    xw, xh = X.shape
    yw = math.floor((xw - kw) / n_stride  + 1)
    yh = math.floor((xh - kh) / n_stride  + 1)
    
    Y = torch.zeros(yw, yh)
    for u in range(yw):
        for v in range(yh):
            xu, xv = u*n_stride, v*n_stride
            selection = X[xu:xu+kw, xv:xv+kh]
            Y[u, v] = (selection*K).sum()
    return Y

In [11]:
image = torch.ones(10, 10)
image[:, 4:6] = 0
image

tensor([[1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1.]])

In [12]:
run_conv(image, torch.FloatTensor([[1, 0, -1]]), n_stride=2)

tensor([[ 0.,  1., -1.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 0.,  1., -1.,  0.]])